# Prepare the DUDE database for analysis

Perform preprocessing on the DUDE database.  Assumes that the DUDE database has been downloaded and placed in the path "data/dude/unprocessed". Here, we expect directories for each target containing 'actives_final.ism' and 'decoys_final.ism'.
It may be beneficial to split the below out of this Jupyter notebook and deploy to HPC or a non interactive environment. To pre-cache descriptors, run cache_mols_from_csv() on the DudePreprocessor object below.

The step may also be skipped by downloading the OpenFEPOPS-DUDE data directory created for analysis of the DUDE diversity set from FigShare at https://doi.org/10.6084/m9.figshare.23951445.v3.  With the data files in this archive unzipped to data/dude/processed/diversity_set/, you can skip the lengthy scoring and use intermediate files to generate the final AUROC table by executing code under the heading "Output summary AUROC results table as shown in the paper"

In [2]:
from fepops.utils import DudePreprocessor
dude_preprocessor=DudePreprocessor()
dude_preprocessor()

data/dude/
Processing the following DUDE targets: []


Preparing targets: 0it [00:00, ?it/s]


# Analyse the DUDE diversity set and obtain mean and standard deviations for each 'standard' set of FEPOPS descriptors

Below, we read in the DUDE diversity set and gather the mean and standard deviation of the produced FEPOPS descriptors and use these values as defaults for scaling (before scoring) all FEPOPS descriptors. Before running this, there should be 8 targets:

* akt1
* ampc
* cp3a4
* cxcr4
* gcr
* hivpr
* hivrt
* kif11

represented by their associated .csv and .db files present in the data/dude/processed/diversity_set/ directory

In [3]:
from pathlib import Path
from tqdm import tqdm
from fepops.fepops_persistent import get_persistent_fepops_storage_object
import numpy as np
dude_diversity_set_path=Path("data/dude/processed/diversity_set/")
diversity_target_files=list(dude_diversity_set_path.glob("dude_target_*.csv"))
descriptors=[]
for diversity_target in diversity_target_files:
    f=get_persistent_fepops_storage_object(diversity_target.with_suffix(".db"))
    print(f"Working on {diversity_target}")
    for (orig_smi, dude_id, chemblid, active_flag, can_smi) in tqdm([l.strip().split(",") for l in open(diversity_target).readlines()[1:] if len(l)>3]):
        status, retrieved_descriptors=f.get_fepops(can_smi)
        if status.value ==1:
            for d in retrieved_descriptors:
                descriptors.append(d)
            
descriptors=np.array(descriptors)
display("Mean:", descriptors.mean(axis=0))
display("Std:", descriptors.std(axis=0))

Working on data/dude/processed/diversity_set/dude_target_kif11.csv


 19%|█▉        | 1335/6966 [00:00<00:02, 2526.54it/s]

100%|██████████| 6966/6966 [00:02<00:00, 2563.43it/s]


Working on data/dude/processed/diversity_set/dude_target_hivrt.csv


100%|██████████| 19229/19229 [00:07<00:00, 2667.88it/s]


...

100%|██████████| 36286/36286 [00:15<00:00, 2320.89it/s]


'Mean:'

array([-0.28971602,  0.5181022 ,  0.37487135,  0.99922747, -0.04187301,
        1.03382471,  0.27407036,  0.99853436,  0.09725517,  1.12824307,
        0.23735556,  0.99882914,  0.35977538,  0.66653514,  0.41238282,
        0.99902545,  5.71261449,  6.37716992,  6.47293777,  6.26134733,
        6.20354385,  6.23201498])

'Std:'

array([0.35110473, 1.00839329, 0.4838859 , 0.02769204, 0.15418035,
       0.86446056, 0.44583626, 0.0381767 , 0.16095862, 0.92079483,
       0.42526185, 0.03413741, 0.35756229, 1.36093993, 0.4921059 ,
       0.0311619 , 1.9668792 , 2.31266486, 2.50699385, 2.41269982,
       2.30018205, 2.31527129])

# Benchmark FEPOPS using the DUDE diversity set
Collect AUROC scores and compare against Morgan 2 and RDKit fingeprints
<hr>

Perform imports, define similarity methods (except OpenFEPOPS which will be defined upon target assessment in order to load cached descriptors)

In [4]:
import numpy as np
from tqdm import tqdm

from dataclasses import dataclass
from typing import Callable, Optional
from pathlib import Path
import pandas as pd
from rdkit import Chem

from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
from sklearn.metrics import roc_auc_score
from fepops import OpenFEPOPS
from fepops.fepops_persistent import get_persistent_fepops_storage_object
from fepops.fepops import GetFepopStatusCode
import json

open_fepops_object=OpenFEPOPS()

@dataclass
class SimilarityMethod:
    name: str
    supports_multiple_candidates: bool
    descriptor_calc_func: Optional[Callable] = None
    descriptor_score_func: Optional[Callable] = None


# OpenFEPOPS will be added to this as a persistent object, reading from a DB of cached
# molecules for each target in the diversity set
similarity_methods = {
    'Morgan 2': SimilarityMethod(
        "Morgan 2",
        False,
        lambda x: AllChem.GetMorganFingerprint(x, 2),
        lambda x, y: DataStructs.TanimotoSimilarity(x,y),
    ),
    'MACCS': SimilarityMethod(
        "MACCS",
        False,
        lambda x: rdMolDescriptors.GetMACCSKeysFingerprint(x),
        lambda x, y: DataStructs.TanimotoSimilarity(x,y),
    ),
    'RDKit': SimilarityMethod(
        "RDKit",
        False,
        lambda x: Chem.RDKFingerprint(x,maxPath=4),
        lambda x, y: DataStructs.TanimotoSimilarity(x,y),
    ),
}

diversity_set_csv_files = list(Path("data/dude/processed/diversity_set/").glob("dude_target_*.csv"))
# Write all CSVs to SMILES files
print(f"Got {len(diversity_set_csv_files)} diversity_set_csv_files : {[f.stem for f in diversity_set_csv_files]}")
for csv_file_path in diversity_set_csv_files:
    pd.read_csv(csv_file_path,sep=",",
        index_col=[0],
        header=0,
        ).loc[:,['rdkit_canonical_smiles', 'DUDEID']].reset_index().to_csv(csv_file_path.with_suffix(".smi"), sep=" ", index=None, header=False)
    

Got 8 diversity_set_csv_files : ['dude_target_kif11', 'dude_target_hivrt', 'dude_target_cxcr4', 'dude_target_ampc', 'dude_target_gcr', 'dude_target_cp3a4', 'dude_target_akt1', 'dude_target_hivpr']


Perform AUROC score calculation

In [5]:
diversity_set_csv_dir = diversity_set_csv_files[0].parent
results_files_derived_from_diversity_set_csv_files = [
    f.parent / Path(f"res_df_{f.stem}.csv") for f in diversity_set_csv_files
]

existing_results_files_derived_from_diversity_set_csv_files = [
    f for f in results_files_derived_from_diversity_set_csv_files if f.exists()
]

if len(existing_results_files_derived_from_diversity_set_csv_files)>0:
    raise FileExistsError(
        f"Found processed results files in the {existing_results_files_derived_from_diversity_set_csv_files[0].parent} directory, remove res_df_dude_target_*.csv\nFound files were {existing_results_files_derived_from_diversity_set_csv_files} \nPlease remove"
    )

for csv_file_path in diversity_set_csv_files:
    auroc_scores_info_df = pd.DataFrame()

    # We replace the OpenFepops similarity object at each new diversity CSV file
    # so that new databases may be loaded for speed of descriptor retrieval.
    ofepops_persistent = get_persistent_fepops_storage_object(
        csv_file_path.with_suffix(".db")
    )
    similarity_methods['OpenFEPOPS'] = SimilarityMethod(
        "OpenFEPOPS",
        True,
        lambda x: ofepops_persistent.get_fepops(x, is_canonical=True),
        ofepops_persistent.calc_similarity,
    )

    df = pd.read_csv(
        csv_file_path,
        sep=",",
        index_col=[0],
        header=0,
    ).reset_index()
    print(df.head())
    descriptors = {k: [] for k in similarity_methods.keys()}
    smiles_list = df['rdkit_canonical_smiles'].tolist()
    labels_list = df['Active'].astype(int).tolist()
    problematic_compound_indexes = []
    for sm_name, sm in similarity_methods.items():
        # Cache all descriptors for each molecular similarity technique, as some mols may be bad and have to be removed
        for smiles_i, smiles in tqdm(
            enumerate(smiles_list),
            desc=f"Caching {sm_name} descriptors for {csv_file_path.stem}",
        ):
            mol_from_smiles = open_fepops_object._mol_from_smiles(smiles)
            if mol_from_smiles is None:
                problematic_compound_indexes.append(smiles_i)
                descriptors[sm_name].append(np.nan)
            else:
                res = sm.descriptor_calc_func(mol_from_smiles)
                if isinstance(res, tuple):
                    if (
                        res[0] == GetFepopStatusCode.FAILED_RETRIEVED_NONE
                        or res[0] == GetFepopStatusCode.FAILED_TO_GENERATE
                        or res[0] == GetFepopStatusCode.FAILED_TO_RETRIEVE
                        or res[1] is None
                    ):
                        print(f"Problem with {smiles}, {res}")
                        problematic_compound_indexes.append(smiles_i)
                        descriptors[sm_name].append(np.nan)
                    else:
                        descriptors[sm_name].append(res[-1])
                else:
                    if res is None:
                        problematic_compound_indexes.append(smiles_i)
                    descriptors[sm_name].append(res)
    # Remove failed molecules from pool of descriptors and labels
    for k, v in descriptors.items():
        descriptors[k] = [
            v[ii] for ii in range(len(v)) if ii not in problematic_compound_indexes
        ]
    labels_list = [
        labels_list[ii]
        for ii in range(len(labels_list))
        if ii not in problematic_compound_indexes
    ]
    auroc_scores = {smn: [] for smn in similarity_methods.keys()}

    info_dict = {}
    for sm_name, sm in similarity_methods.items():
        # Remove entries which did not return a mol
        info_dict['target'] = csv_file_path.stem.replace("dude_target_", "")
        info_dict['similarity_method'] = sm_name
        info_dict['smiles_count'] = len(smiles_list)
        info_dict['actives_count'] = np.sum(labels_list)
        info_dict['failed_smiles'] = len(problematic_compound_indexes)
        info_dict['failed_active_smiles'] = len(
            [ft for ft in problematic_compound_indexes if labels_list[ft] == 1]
        )
        for active_i in tqdm(
            np.argwhere(np.array(labels_list) == 1).flatten(),
            desc=f"Assessing active recall (AUROC) for {sm.name}",
        ):
            if sm.supports_multiple_candidates:
                scores = np.array(
                    sm.descriptor_score_func(
                        descriptors[sm_name][active_i], descriptors[sm_name]
                    ),
                    dtype=float,
                ).flatten()
            else:
                scores = np.array(
                    [
                        sm.descriptor_score_func(
                            descriptors[sm_name][active_i],
                            descriptors[sm_name][smiles_i],
                        )
                        for smiles_i in range(len(descriptors[sm_name]))
                    ],
                    dtype=float,
                ).flatten()

            auroc_scores[sm_name].append(
                roc_auc_score(
                    np.array(labels_list)[np.argwhere(~np.isnan(scores))],
                    scores[np.argwhere(~np.isnan(scores))],
                )
            )
        info_dict['average_auroc_score'] = np.mean(auroc_scores[sm_name])
        info_dict['median_auroc_score'] = np.median(auroc_scores[sm_name])
        info_dict['q1_auroc_score'] = np.percentile(auroc_scores[sm_name], 0.25)
        info_dict['q3_auroc_score'] = np.percentile(auroc_scores[sm_name], 0.75)
        auroc_scores_info_df = pd.concat(
            [auroc_scores_info_df, pd.DataFrame.from_dict({k:[v] for k,v in info_dict.items()})], ignore_index=True, axis=0
        )
    print("Writing to ", csv_file_path.parent)

    json.dump(
        auroc_scores,
        open(csv_file_path.parent / Path(f"res_scores_{csv_file_path.stem}.json"), "w"),
    )
    auroc_scores_info_df.to_csv(
        csv_file_path.parent / Path(f"res_df_{csv_file_path.stem}.csv")
    )
print(auroc_scores_info_df)

                                              SMILES  DUDEID      CHEMBLID  \
0  CN(C)CCC[C@@]3(c1ccccc1)CN(c2cc(C)ccc2F)N=C3C(...  415211  CHEMBL247044   
1  CC(=O)C3=NN1[C@@H](COc2ccc(F)cc12)[C@@]3(CCCN4...  414282  CHEMBL398478   
2  N[C@@H](C1CC1)C(=O)N3CC(c2cc(F)ccc2F)=C[C@H]3c...  344118  CHEMBL206115   
3  CC(=O)C3=NN1[C@@H](COc2ccc(F)cc12)[C@@]3(CCCN)...  414288  CHEMBL250127   
4  CC(=O)N2N=C(c1cc(F)ccc1F)C[C@@]2(C[C@H](F)CN)c...  402131  CHEMBL400042   

   Active                             rdkit_canonical_smiles  
0       1  CC(=O)C1=NN(c2cc(C)ccc2F)C[C@@]1(CCCN(C)C)c1cc...  
1       1  CC(=O)C1=NN2c3cc(F)ccc3OC[C@H]2[C@@]1(CCCN1CCO...  
2       1  N[C@H](C(=O)N1CC(c2cc(F)ccc2F)=C[C@H]1c1cccc(O...  
3       1  CC(=O)C1=NN2c3cc(F)ccc3OC[C@H]2[C@@]1(CCCN)c1c...  
4       1  CC(=O)N1N=C(c2cc(F)ccc2F)C[C@@]1(C[C@H](F)CN)c...  


Caching Morgan 2 descriptors for dude_target_kif11: 506it [00:00, 5058.56it/s]

Caching Morgan 2 descriptors for dude_target_kif11: 6966it [00:01, 4818.02it/s]
Caching MACCS descriptors for dude_target_kif11: 6966it [00:05, 1237.28it/s]
Caching RDKit descriptors for dude_target_kif11: 6966it [00:02, 3479.81it/s]
Caching OpenFEPOPS descriptors for dude_target_kif11: 4359it [00:01, 2493.69it/s]

Problem with Cc1ccccc1NC(=O)[C@@H]1[C@H]2/C=C/CCCC[C@H]21, (<GetFepopStatusCode.FAILED_RETRIEVED_NONE: 4>, None)


Caching OpenFEPOPS descriptors for dude_target_kif11: 6966it [00:02, 2458.32it/s]
Assessing active recall (AUROC) for OpenFEPOPS: 100%|██████████| 116/116 [00:37<00:00,  3.13it/s]


...

Problem with C=CO, (<GetFepopStatusCode.FAILED_RETRIEVED_NONE: 4>, None)


Caching OpenFEPOPS descriptors for dude_target_hivpr: 26071it [00:11, 2233.16it/s]

Problem with CC(C)=C/C=C/[C@H](C)[C@H]1CC[C@@]2(C)C[C@@H]3C(C)=CC(=O)[C@@H]3/C(C(=O)[O-])=C/C[C@@H]12, (<GetFepopStatusCode.FAILED_RETRIEVED_NONE: 4>, None)


Caching OpenFEPOPS descriptors for dude_target_hivpr: 36286it [00:16, 2152.21it/s]
Assessing active recall (AUROC) for OpenFEPOPS: 100%|██████████| 535/535 [14:09<00:00,  1.59s/it]

Writing to  data/dude/processed/diversity_set
  target similarity_method  smiles_count  actives_count  failed_smiles  \
0  hivpr          Morgan 2         36286            535              2   
1  hivpr             MACCS         36286            535              2   
2  hivpr             RDKit         36286            535              2   
3  hivpr        OpenFEPOPS         36286            535              2   

   failed_active_smiles  average_auroc_score  median_auroc_score  \
0                     1             0.779684            0.808723   
1                     1             0.681231            0.700411   
2                     1             0.759303            0.781645   
3                     1             0.678182            0.704364   

   q1_auroc_score  q3_auroc_score  
0        0.238002        0.402323  
1        0.283952        0.315156  
2        0.309755        0.369676  
3        0.334860        0.351404  


# Output summary AUROC results table as shown in the paper

In [8]:
import pandas as pd
from pathlib import Path
display(pd.concat([pd.read_csv(f) for f in Path("data/dude/processed/diversity_set/").glob("res_df_dude_target_*.csv")]).pivot(values='average_auroc_score',index='target', columns=['similarity_method'],)[['Morgan 2', 'RDKit', 'MACCS', 'OpenFEPOPS']])

similarity_method,Morgan 2,RDKit,MACCS,OpenFEPOPS
target,,,,
akt1,0.835717,0.833538,0.741306,0.831254
ampc,0.783629,0.659837,0.673310,0.639413
cp3a4,0.602774,0.613335,0.581545,0.647462
cxcr4,0.697226,0.592485,0.854251,0.899027
gcr,0.670082,0.708346,0.665830,0.616228
hivpr,0.779684,0.759303,0.681231,0.678182
hivrt,0.651023,0.660070,0.669954,0.582240
kif11,0.763057,0.672469,0.667852,0.713142


Output an extended table with information on failed molecules etc

In [9]:
import pandas as pd
from pathlib import Path
display(pd.concat([pd.concat([pd.read_csv(f) for f in Path("data/dude/processed/diversity_set/").glob("res_df_dude_target_*.csv")]).pivot(values='average_auroc_score',index='target', columns=['similarity_method'],),pd.concat([pd.read_csv(f) for f in Path("data/dude/processed/diversity_set/").glob("res_df*.csv")]).query("similarity_method=='OpenFEPOPS'")[['target','smiles_count','actives_count','failed_smiles','failed_active_smiles']].set_index("target")], axis=1))

,MACCS,Morgan 2,OpenFEPOPS,RDKit,smiles_count,actives_count,failed_smiles,failed_active_smiles
target,,,,,,,,
akt1,0.741306,0.835717,0.831254,0.833538,16743,293,0,0
ampc,0.673310,0.783629,0.639413,0.659837,2898,48,1,0
cp3a4,0.581545,0.602774,0.647462,0.613335,11970,170,4,0
cxcr4,0.854251,0.697226,0.899027,0.592485,3446,40,0,0
gcr,0.665830,0.670082,0.616228,0.708346,15258,258,10,0
hivpr,0.681231,0.779684,0.678182,0.759303,36286,535,2,1
hivrt,0.669954,0.651023,0.582240,0.660070,19229,338,8,0
kif11,0.667852,0.763057,0.713142,0.672469,6966,116,1,0
